<a href="https://colab.research.google.com/github/vithushanms/convolutional-neural-network-demo/blob/main/%5BCNNs%5D_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from tensorflow.keras import datasets, utils

In [ ]:
#
(x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()